## Neighbourhood Identification
Code source: https://github.com/nolanlab/NeighborhoodCoordination/blob/master/Neighborhoods/Neighborhood%20Identification.ipynb

Identify cellular neighbourhoods in the archived/all_data.csv

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
import os

In [ ]:
def get_windows(job,n_neighbors):
    """
    For each region and each individual cell in dataset, return the indices of the nearest neighbors.
    
    'job:  meta data containing the start time,index of region, region name, indices of region in original dataframe
    n_neighbors:  the number of neighbors to find for each cell
    """
    start_time,idx,tissue_name,indices = job
    job_start = time.time()
    
    print ("Starting:", str(idx+1)+'/'+str(len(exps)),': ' + exps[idx])

    tissue = tissue_group.get_group(tissue_name)
    to_fit = tissue.loc[indices][[X,Y]].values

#     fit = NearestNeighbors(n_neighbors=n_neighbors+1).fit(tissue[[X,Y]].values)
    fit = NearestNeighbors(n_neighbors=n_neighbors).fit(tissue[[X,Y]].values)
    m = fit.kneighbors(to_fit)
#     m = m[0][:,1:], m[1][:,1:]
    m = m[0], m[1]
    

    #sort_neighbors
    args = m[0].argsort(axis = 1)
    add = np.arange(m[1].shape[0])*m[1].shape[1]
    sorted_indices = m[1].flatten()[args+add[:,None]]

    neighbors = tissue.index.values[sorted_indices]
   
    end_time = time.time()
   
    print ("Finishing:", str(idx+1)+"/"+str(len(exps)),": "+ exps[idx],end_time-job_start,end_time-start_time)
    return neighbors.astype(np.int32)

### Fill In:
- ks = individual window sizes to collect.  
    -This accelerates the process of trying different window sizes by collecting them all in one step.
- path_to_data:  file path for csv file with x,y cluster information
- X:  column to use for X location
- Y:  column to use for Y location
- reg:  column to use for region (should be different for every tissue)
- file_type('csv','pickle'):  file type
- cluster_col : column name of cell type cluster to be used for neighborhoods
- keep_cols:  columns to transfer from original dataframe to dataframe with windows and neighborhoods

In [ ]:
ks = [5,10,20] # k=5 means it collects 5 nearest neighbors for each center cell
path_to_data = os.path.join('..', 'outputs', 'dr_clstr.csv')
X = 'Centroid X µm'
Y = 'Centroid Y µm'
reg = 'Parent'
file_type = 'csv'

cluster_col = 'Cluster'
keep_cols = [X,Y,reg,cluster_col, 'Image', 'Object ID', 'Timepoint', 'Phenotype']
save_path = os.path.join('..', 'outputs', 'neighbourhood_id.csv')

In [ ]:
#read in data and do some quick data rearrangement
n_neighbors = max(ks)
assert (file_type=='csv' or file_type =='pickle') #


if file_type == 'pickle':
    cells = pd.read_pickle(path_to_data)
if file_type == 'csv':
    cells = pd.read_csv(path_to_data)

cells = pd.concat([cells,pd.get_dummies(cells[cluster_col])],axis=1)


#cells = cells.reset_index() #Uncomment this line if you do any subsetting of dataframe such as removing dirt etc or will throw error at end of next next code block (cell 6)

sum_cols = cells[cluster_col].unique()
values = cells[sum_cols].values

In [ ]:
#find windows for each cell in each tissue region
tissue_group = cells[[X,Y,reg]].groupby(reg)
exps = list(cells[reg].unique())
tissue_chunks = [(time.time(),exps.index(t),t,a) for t,indices in tissue_group.groups.items() for a in np.array_split(indices,1)] 
tissues = [get_windows(job,n_neighbors) for job in tissue_chunks]

In [ ]:
#for each cell and its nearest neighbors, reshape and count the number of each cell type in those neighbors.
out_dict = {}
for k in ks:
    for neighbors,job in zip(tissues,tissue_chunks):

        chunk = np.arange(len(neighbors))#indices
        tissue_name = job[2]
        indices = job[3]
        window = values[neighbors[chunk,:k].flatten()].reshape(len(chunk),k,len(sum_cols)).sum(axis = 1)
        out_dict[(tissue_name,k)] = (window.astype(np.float16),indices)

#concatenate the summed windows and combine into one dataframe for each window size tested.
windows = {}
for k in ks:
   
    window = pd.concat([pd.DataFrame(out_dict[(exp,k)][0],index = out_dict[(exp,k)][1].astype(int),columns = sum_cols) for exp in exps],axis=0)
    window = window.loc[cells.index.values]
    window = pd.concat([cells[keep_cols],window],axis=1)
    windows[k] = window

### Fill In:
- k:  window_size (needs to be in ks (saved above))
- n_neighborhoods:  number of neighborhoods to find

In [ ]:
k = 10
n_neighborhoods = 10
neighborhood_name = "neighborhood"+str(k)
k_centroids = {}

In [ ]:
windows2 = windows[10]
# windows2[cluster_col] = cells[cluster_col]

km = MiniBatchKMeans(n_clusters = n_neighborhoods,random_state=0)

labelskm = km.fit_predict(windows2[sum_cols].values)
k_centroids[k] = km.cluster_centers_
cells['neighborhood10'] = labelskm
cells[neighborhood_name] = cells[neighborhood_name].astype('category')
#['reg064_A','reg066_A','reg018_B','reg023_A']

In [ ]:
# cell_order = ['tumor cells', 'CD11c+ DCs', 'tumor cells / immune cells',
#        'smooth muscle', 'lymphatics', 'adipocytes', 'undefined',
#        'CD4+ T cells CD45RO+', 'CD8+ T cells', 'CD68+CD163+ macrophages',
#        'plasma cells', 'Tregs', 'immune cells / vasculature', 'stroma',
#        'CD68+ macrophages GzmB+', 'vasculature', 'nerves',
#        'CD11b+CD68+ macrophages', 'granulocytes', 'CD68+ macrophages',
#        'NK cells', 'CD11b+ monocytes', 'immune cells',
#        'CD4+ T cells GATA3+', 'CD163+ macrophages', 'CD3+ T cells',
#        'CD4+ T cells', 'B cells']

In [ ]:
# this plot shows the types of cells (ClusterIDs) in the different niches (0-7)
k_to_plot = 10
niche_clusters = (k_centroids[k_to_plot])
tissue_avgs = values.mean(axis = 0)
fc = np.log2(((niche_clusters+tissue_avgs)/(niche_clusters+tissue_avgs).sum(axis = 1, keepdims = True))/tissue_avgs)
fc = pd.DataFrame(fc,columns = sum_cols)

s=sns.clustermap(fc.loc[[0,2,3,4,5,6,7,8,9]], vmin =-3,vmax = 3,cmap = 'bwr',row_cluster = False)
# adding titles
s.figure.suptitle('Cell Clusters in Neighbourhoods', fontsize=16)
s.ax_heatmap.set_xlabel('Cluster', fontsize=14)
s.ax_heatmap.set_ylabel('Cellular Neighbourhood', fontsize=14)

# s=sns.clustermap(fc.loc[[0,2,3,4,5,6,7,8,9],cell_order], vmin =-3,vmax = 3,cmap = 'bwr',row_cluster = False)
# s.savefig("raw_figs/celltypes_perniche_10.pdf")

In [ ]:
# cells['neighborhood10'] = cells['neighborhood10'].astype('category')
# sns.lmplot(data = cells[cells['Timepoint']=='baseline'],x = 'CENTERX',y='CENTERY',hue = 'neighborhood10',palette = 'bright',height = 8,col = reg,col_wrap = 10,fit_reg = False)

In [ ]:
cells['neighborhood10'] = cells['neighborhood10'].astype('category')
# for each sample, plot the xy coordinates
grouped = cells.groupby('Parent')
for name, group in grouped:
    # plot the xy coordinates 
    if group.iloc[0]['Timepoint'] == 'baseline':
        pointsize = 3
        legendpoint = 5
    else:
        pointsize = 40
        legendpoint = 2
    f, ax = plt.subplots(figsize=(10, 10))

    sns.scatterplot(
        x="Centroid X µm", 
        y="Centroid Y µm",
        hue="neighborhood10",
        legend="full",
        palette="rainbow",
        data=group,
        ax=ax,
        s=pointsize
    ).set(title=f'Sample {name}')

    sns.despine()
    ax.legend(title='Cellular Neighbourhood', bbox_to_anchor=(1.05, 1), loc=2, markerscale=legendpoint, borderaxespad=0.)
    plt.show()

In [ ]:
markers = ['DAPI', 'CD44', 'HLA-DR', 'CD4', 'IFNG', 'Ki67', 'CD107a', 'CD45', 'CD20', 'CD40', 'CD8', 'Pan-Cytokeratin', 'CD68', 'HLA-A', 'CD79a', 'CD45RO', 'CD21', 'CD11c', 'HLA-E', 'IDO1', 'CD14', 'CD56', 'VISTA', 'FOXP3', 'Granzyme B', 'PCNA', 'T-bet/TBX21', 'PD-L1', 'TOX', 'PD-1', 'CD38', 'ICOS', 'CD39', 'LAG3', 'TCF-1', 'CD3e']
metadata = keep_cols + 'neighboorhood10'

In [ ]:
cells.head()

In [ ]:
heatmap_data = cells.groupby('neighborhood10')[markers].mean().reset_index()
heatmap_data = heatmap_data.set_index('neighborhood10')

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='YlGnBu', fmt=".2f")
plt.xlabel('Biomarkers')
plt.ylabel('Cellular Neighborhood')
plt.title('Heatmap of Biomarkers by Neighborhood')

In [ ]:
# figure out aggfunc for this one, but basically a heatmap of timepoint by neighbourhood
heatmap_data = cells.pivot_table(index='neighborhood10', columns='Timepoint', aggfunc='size', fill_value=0)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=True, fmt="d")  # 'fmt="d"' for integer formatting
plt.xlabel('Timepoint')
plt.ylabel('Cellular Neighborhood')
plt.title('Heatmap of Timepoint by Neighborhood')
plt.show()

In [ ]:
save_path = os.path.join('..', 'outputs', 'cells2.csv')
cells.to_csv(save_path, index=False)